# Generating isochrones around pharmacies in New York State

This notebook uses OpenRouteService to generate isochrones within a 10-minute drive around open pharmacies on June 21, 2017 and June 21, 2022. This data will be used to measure the population living inside and outside that area, calculations done in a separate notebook.

New York City was excluded from the analysis since its residents largely walk or use public transit to get around. 

In [1]:
import requests
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import time
from getpass import getpass
pd.set_option("display.max_columns", None)

In [ ]:
api_key = getpass('Enter OpenRouteService API Key: ')

## Isochrones for open pharmacies as of June 21, 2022

In [2]:
# File containing pharmacies outside NYC and in Staten Island 
df = pd.read_csv('data/driving_nys_pharmacies.csv')

In [6]:
# Filtering out Staten Island to focus just on upstate NY 
df = df.query('county != "Richmond"')

In [7]:
def get_pair(long, lat):
    return [long, lat]
    
df['long_lat_pair'] = df.apply(lambda x: get_pair(x.displayLongitude, x.displayLatitude), axis=1)

In [10]:
len(list(df.long_lat_pair))

2443

In [ ]:
location_list = list(df.long_lat_pair)
geoframes = []
for i in tqdm(range(0, len(location_list), 5)):
    if i % 20 == 0:
        time.sleep(60)
    body = {"locations":location_list[i:i+5],"range_type":"time", "range":[600]} #changed range from 900 to reduce from 15 min to 10

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car', json=body, headers=headers)
    try:
        geoframe = gpd.GeoDataFrame.from_features(call.json())
    except:
        print(i, call.json())
        break
    geoframes.append(geoframe)

In [13]:
geoframe_concat = gpd.GeoDataFrame(pd.concat(geoframes, ignore_index=True))

In [20]:
geoframe_concat = geoframe_concat.drop_duplicates('geometry')

In [21]:
geoframe_concat['center'] = geoframe_concat['center'].apply(lambda x: ' '.join(str(x)))

/Users/jmingram/.pyenv/versions/3.10.0/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [22]:
geoframe_concat.to_file('data/active_driving_isochrones_10min.geojson', driver='GeoJSON')

## Isochrones for pharmacies open as of June 21, 2017

In [24]:
api_key = getpass('Enter API Key: ')

In [25]:
all_pharmacies = pd.read_csv('data/pharmacies_geocoded_6_28.csv', parse_dates=['date_first_registered',
                                                                    'registration_begins',
                                                                    'registration_ends'])

In [26]:
exclude = ['Queens', 'New York', 'Bronx', 'Kings', 'Richmond']

In [27]:
before_frame = all_pharmacies[(all_pharmacies.date_first_registered <= "2017-06-21") &\
               (all_pharmacies.registration_ends > "2017-06-21") &\
               (all_pharmacies.county.isin(exclude)==False)]

In [28]:
before_frame.to_csv('data/driving_pharmacies_2017_snapshot_no_nyc.csv', index=False)

In [29]:
before_frame.shape

(2386, 32)

In [30]:
before_frame['long_lat_pair'] = before_frame.apply(lambda x: get_pair(x.displayLongitude, x.displayLatitude), axis=1)

/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_47319/2872414449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_frame['long_lat_pair'] = before_frame.apply(lambda x: get_pair(x.displayLongitude, x.displayLatitude), axis=1)


In [ ]:
before_location_list = list(before_frame.long_lat_pair)
before_geoframes = []
for i in tqdm(range(0, len(before_location_list), 5)):
    if i % 100 == 0:
        time.sleep(30)
    body = {"locations":before_location_list[i:i+5],"range_type":"time", "range":[900]}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car', json=body, headers=headers)
    try:
        before_geoframe = gpd.GeoDataFrame.from_features(call.json())
    except:
        print(i, call.json())
        break
    before_geoframes.append(before_geoframe)

In [33]:
before_geoframe_concat = gpd.GeoDataFrame(pd.concat(before_geoframes, ignore_index=True))

In [34]:
before_geoframe_concat = before_geoframe_concat.drop_duplicates('geometry')

In [35]:
before_geoframe_concat['center'] = before_geoframe_concat['center'].apply(lambda x: ' '.join(str(x)))

In [36]:
before_geoframe_concat.to_file('data/2017_driving_isochrones_10min.geojson', driver='GeoJSON')